# Подбор оптимальных параметров тренировки аффиксного тэггера.

### Гудков В.В.  1 курс магистратура СПбГУ, кафедра математической лингвистики

In [1]:
from nltk4russian.util import read_corpus_to_nltk
from nltk import sequential
import statistics

In [2]:
#Статистика по словам
def words_stats(sents, name = " "):
    
    words = [] #Тут будут все слова корпуса
    
    for sent in sents:
        for word_pos in sent:
            words.append(word_pos[0])
                
    words_lens = [len(word) for word in words] #Длины всех слов в корпусе
    mean_words = round(statistics.mean(words_lens), 2)
    less_than_4 = len([i for i in words_lens if i < 4])
    more_than_8 = len([j for j in words_lens if j > 8])
    
    print("Средняя длина слова в корпусе " + name + " = " + str(mean_words))
    print("Количество слов меньше 4-х символов в корпусе " + name + " = " + str(less_than_4) + " (" + 
          str(round(less_than_4 / len(words)*100, 2)) + "%)")
    print("Количество слов больше 8-и символов в корпусе " + name + " = " + str(more_than_8)+ " (" + 
          str(round(more_than_8/ len(words)*100, 2)) + "%)")
    
    
#Перебор всех вариантов тэггеров для данного датасета (длины суффиксов и минимальных стемм) 
def go_thru_loop(train, test):
    for i in range(-5,-1):
        for j in range(4):
            if -i+j > 6:
                break
            else:
                print("Тренировка тэггера...")
                print("Длина суффикса ", i*-1)
                print("Минимальная стемма ", j)
                tagger = sequential.AffixTagger(train=train, affix_length=i,
                                                          min_stem_length=j, verbose=False)
                print("Результат:", round(tagger.evaluate(test), 3)*100, "%")
                print('-----------------------------')

# Удаление коротких словоформ и других знаков из корпуса
def rmv_irr_tokens(sents):
    
    print('--------------------')
    print("Количество словоупотреблений до чистки: " + str(sum(len(i) for i in sents)))
    num_of_irr = 0
    
    for i in range(6):
        for sent in sents:
            for word_pos in sent:
                pos = list(word_pos)[1].split(',')[0]
                if pos in ['PNCT', 'NUMR', 'PRTF', 'PRCL', 'PREP', 'UNKN', 'CONJ', 'PRTS',
                                  'sing', 'plur', 'neut', '', 'masc', 'INTJ']:
                    sent.remove(word_pos)
                    num_of_irr += 1
                elif pos is 'ADJF' and len(pos) <= 4:
                    sent.remove(word_pos)
                    num_of_irr += 1
            if len(sent) == 0:
                sents.remove(sent)

    print("Количество удаленных токенов: " + str(num_of_irr))
    print("Количество словоупотреблений после чистки: " + str(sum(len(i) for i in sents)))
    print('--------------------')
    
#Преобразование морф корпуса в POS корпус
def full_morph_to_pos(sents):
    
    sent_index = 0
    for sent in sents:
        word_pos_index = 0
        for word_pos in sent:
            word_pos = list(word_pos)
            sents[sent_index][word_pos_index] = list(sents[sent_index][word_pos_index])
            sents[sent_index][word_pos_index][1] = word_pos[1].split(',')[0]
            sents[sent_index][word_pos_index] = tuple(sents[sent_index][word_pos_index])
            word_pos_index += 1
        sent_index += 1
        

#Разметка своих слов натренированными теггерами из списка        
def apply_tagger_to_list(tagger, word_list, counter = 0):

    print("----------")
    for tok, tag in tagger.tag(word_list):
        if tag is None:
            counter += 1
        print("(%s, %s), " % (tok, tag))
    print("----------")
    print("Неопознанных слов: %d" % counter)
    print()

In [3]:
with open('media_train.tab', encoding = 'utf-8', mode='r') as f:
    sents_train = list(read_corpus_to_nltk(f))

with open('media_test.tab', encoding = 'utf-8', mode='r') as r:
    sents_test = list(read_corpus_to_nltk(r))
    
words_stats(sents_train, name = "sents_train")
print('--------------------------')
words_stats(sents_test, name = "sents_test")

print("|||||||||||||||||||||||||")
go_thru_loop(sents_train, sents_test) #Учим без предобработки

Средняя длина слова в корпусе sents_train = 5.02
Количество слов меньше 4-х символов в корпусе sents_train = 40151 (43.3%)
Количество слов больше 8-и символов в корпусе sents_train = 17502 (18.88%)
--------------------------
Средняя длина слова в корпусе sents_test = 5.12
Количество слов меньше 4-х символов в корпусе sents_test = 8178 (42.66%)
Количество слов больше 8-и символов в корпусе sents_test = 3804 (19.85%)
|||||||||||||||||||||||||
Тренировка тэггера...
Длина суффикса  5
Минимальная стемма  0
Результат: 34.1 %
-----------------------------
Тренировка тэггера...
Длина суффикса  5
Минимальная стемма  1
Результат: 28.599999999999998 %
-----------------------------
Тренировка тэггера...
Длина суффикса  4
Минимальная стемма  0
Результат: 37.5 %
-----------------------------
Тренировка тэггера...
Длина суффикса  4
Минимальная стемма  1
Результат: 33.5 %
-----------------------------
Тренировка тэггера...
Длина суффикса  4
Минимальная стемма  2
Результат: 28.4 %
---------------------

In [4]:
with open('media_train.tab', encoding = 'utf-8', mode='r') as f:
    sents_train = list(read_corpus_to_nltk(f))

with open('media_test.tab', encoding = 'utf-8', mode='r') as r:
    sents_test = list(read_corpus_to_nltk(r))

rmv_irr_tokens(sents_train) #Удаляем все слова, кроме {'GRND', 'ADJS', 'VERB', 'PRED', 'INFN', 'COMP', 'ADVB', 'ADJF', 'NOUN'}
rmv_irr_tokens(sents_test)

words_stats(sents_train, name = "sents_train") #Длины слов после чистки
print('--------------------------')
words_stats(sents_test, name = "sents_test")

print("|||||||||||||||||||||||||")
go_thru_loop(sents_train, sents_test) #После удаления нерелевантных слов и знаков (результаты улучшились)

--------------------
Количество словоупотреблений до чистки: 92720
Количество удаленных токенов: 39062
Количество словоупотреблений после чистки: 53658
--------------------
--------------------
Количество словоупотреблений до чистки: 19168
Количество удаленных токенов: 7819
Количество словоупотреблений после чистки: 11349
--------------------
Средняя длина слова в корпусе sents_train = 7.19
Количество слов меньше 4-х символов в корпусе sents_train = 5929 (11.05%)
Количество слов больше 8-и символов в корпусе sents_train = 16221 (30.23%)
--------------------------
Средняя длина слова в корпусе sents_test = 7.3
Количество слов меньше 4-х символов в корпусе sents_test = 1216 (10.71%)
Количество слов больше 8-и символов в корпусе sents_test = 3584 (31.58%)
|||||||||||||||||||||||||
Тренировка тэггера...
Длина суффикса  5
Минимальная стемма  0
Результат: 54.300000000000004 %
-----------------------------
Тренировка тэггера...
Длина суффикса  5
Минимальная стемма  1
Результат: 46.1 %
-------

In [5]:
with open('media_train.tab', encoding = 'utf-8', mode='r') as f:
    sents_train = list(read_corpus_to_nltk(f))

with open('media_test.tab', encoding = 'utf-8', mode='r') as r:
    sents_test = list(read_corpus_to_nltk(r))

rmv_irr_tokens(sents_train)
rmv_irr_tokens(sents_test)

full_morph_to_pos(sents_train) #Теперь тренируем POS тэггер
full_morph_to_pos(sents_test)

pos_list = []
for sent in sents_train:
    for word_pos in sent:
        pos = word_pos[1]
        pos_list.append(pos)
print('--------------------')
print("Оставшиеся метки: ", set(pos_list))
print('--------------------')

go_thru_loop(sents_train, sents_test)

--------------------
Количество словоупотреблений до чистки: 92720
Количество удаленных токенов: 39062
Количество словоупотреблений после чистки: 53658
--------------------
--------------------
Количество словоупотреблений до чистки: 19168
Количество удаленных токенов: 7819
Количество словоупотреблений после чистки: 11349
--------------------
--------------------
Оставшиеся метки:  {'ADJS', 'ADJF', 'INFN', 'PRED', 'COMP', 'GRND', 'NOUN', 'ADVB', 'VERB'}
--------------------
Тренировка тэггера...
Длина суффикса  5
Минимальная стемма  0
Результат: 70.0 %
-----------------------------
Тренировка тэггера...
Длина суффикса  5
Минимальная стемма  1
Результат: 59.699999999999996 %
-----------------------------
Тренировка тэггера...
Длина суффикса  4
Минимальная стемма  0
Результат: 81.5 %
-----------------------------
Тренировка тэггера...
Длина суффикса  4
Минимальная стемма  1
Результат: 74.2 %
-----------------------------
Тренировка тэггера...
Длина суффикса  4
Минимальная стемма  2
Резул

In [7]:
# Тест на своем списке

cool_list  = ["Хурма", "термос", "глокая", "куздра", "самовар", "варенька", "этот", "соль"]

with open('media_train.tab', encoding = 'utf-8', mode='r') as f:
    sents_train = list(read_corpus_to_nltk(f))

#Полный морф разбор
rmv_irr_tokens(sents_train)
test_full_tagger = sequential.AffixTagger(train=sents_train, affix_length=-3, min_stem_length=0, verbose=False)
apply_tagger_to_list(test_full_tagger, cool_list)

#POS тэггинг
full_morph_to_pos(sents_train)
test_pos_tagger = sequential.AffixTagger(train=sents_train, affix_length=-3, min_stem_length=0, verbose=False)
apply_tagger_to_list(test_pos_tagger, cool_list)

--------------------
Количество словоупотреблений до чистки: 92720
Количество удаленных токенов: 39062
Количество словоупотреблений после чистки: 53658
--------------------
----------
(Хурма, NOUN,femn,inan,sing,nomn), 
(термос, NOUN,masc,inan,sing,nomn), 
(глокая, ADJF,femn,sing,nomn), 
(куздра, NOUN,masc,anim,sing,gent), 
(самовар, NOUN,masc,inan,sing,accs), 
(варенька, NOUN,masc,anim,sing,nomn), 
(этот, ADJF,Apro,masc,sing,accs,inan), 
(соль, NOUN,femn,inan,sing,accs), 
----------
Неопознанных слов: 0

----------
(Хурма, NOUN), 
(термос, NOUN), 
(глокая, ADJF), 
(куздра, NOUN), 
(самовар, NOUN), 
(варенька, NOUN), 
(этот, ADJF), 
(соль, NOUN), 
----------
Неопознанных слов: 0

